# CatBoost Regressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
import pandas as pd
dataset = pd.read_csv('insurance.csv')

In [ ]:
dataset.head()

### Checking missing data

In [ ]:
dataset.info()

### Handling categorical variables

Sex column

In [ ]:
dataset['sex'].unique()

In [ ]:
dataset['sex'] = dataset['sex'].apply(lambda x: 0 if x == 'female' else 1)

In [ ]:
dataset.head()

Smoker column

In [ ]:
dataset['smoker'].unique()

In [ ]:
dataset['smoker'] = dataset['smoker'].apply(lambda x: 0 if x == 'no' else 1)

In [ ]:
dataset.head()

Region column

In [ ]:
dataset['region'].unique()

In [ ]:
region_dummies = pd.get_dummies(dataset['region'], drop_first = True)

In [ ]:
region_dummies

In [ ]:
dataset = pd.concat([region_dummies, dataset], axis = 1)

In [ ]:
dataset.head()

In [ ]:
dataset.drop(['region'], axis = 1, inplace = True)

In [ ]:
dataset.head()

### Creating the Training Set and the Test Set

Getting the inputs and output

In [ ]:
X = dataset.iloc[:, :-1].values

In [ ]:
y = dataset.iloc[:, -1].values

In [ ]:
X

In [ ]:
y

Getting the Training Set and the Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Part 2 - Building and training the model

### Building the model

In [ ]:
!pip install catboost

In [ ]:
import catboost as cb
model = cb.CatBoostRegressor()

### Training the model

In [ ]:
model.fit(X_train, y_train)

### Inference

In [ ]:
y_pred = model.predict(X_test)

## Part 3: Evaluating the model

### R-Squared

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

In [ ]:
r2

### Adjusted R-Squared

In [ ]:
k = X_test.shape[1]
n = len(X_test)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

In [ ]:
adj_r2

### k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
r2s = cross_val_score(estimator = model,
                      X = X,
                      y = y,
                      scoring = 'r2',
                      cv = 10)
print("R-Squared: {:.2f} %".format(r2s.mean()*100))
print("Standard Deviation: {:.2f} %".format(r2s.std()*100))

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'learning_rate': [0.008,0.009,0.01],
               'depth': [4,7,10],
               'l2_leaf_reg': [2,6,10],
               'random_strength': [0,5,10]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10)
grid_search.fit(X, y)
best_r2 = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best R-Squared: {:.2f} %".format(best_r2*100))
print("Best Parameters:", best_parameters)